In [3]:
import pandas as pd
import numpy as np

# ===============================
# 1. 데이터 로드
# ===============================
population_file = '2025_서울시_구별_인구.csv'
crime_file = '서울시_25개구_5대범죄_연도별통합.csv'

# 인코딩을 시도하며 읽기
try:
    pop_df = pd.read_csv(population_file, encoding='cp949')
except:
    try:
        pop_df = pd.read_csv(population_file, encoding='euc-kr')
    except:
        pop_df = pd.read_csv(population_file, encoding='utf-8-sig')

try:
    crime_df = pd.read_csv(crime_file, encoding='cp949')
except:
    try:
        crime_df = pd.read_csv(crime_file, encoding='euc-kr')
    except:
        crime_df = pd.read_csv(crime_file, encoding='utf-8-sig')

print("데이터 로드 완료")

# ===============================
# 2. 인구 데이터 전처리 (2015-2024년만, 2025년과 2014년 이전 제외)
# ===============================
# 첫 행이 헤더인 경우 제거
if '세대' in str(pop_df.iloc[0]['동별(2)']):
    pop_df = pop_df.iloc[1:].reset_index(drop=True)

# 범죄 데이터가 있는 연도만 사용 (2015-2024)
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
population_data = []

for idx, row in pop_df.iterrows():
    region = row['동별(2)']

    # 합계, 총계, 소계 등 제외
    if pd.isna(region) or region == '' or '합계' in str(region) or '총계' in str(region) or '소계' in str(region):
        continue

    for year in years:
        # 해당 연도의 컬럼 찾기
        year_cols = [col for col in pop_df.columns if str(year) in str(col)]

        if year_cols:
            # 첫 번째 또는 두 번째 컬럼 사용 (세대가 아닌 인구수)
            col_name = year_cols[0] if len(year_cols) == 1 else year_cols[1]

            if pd.notna(row[col_name]):
                pop_value = row[col_name]
                # 콤마 제거
                if isinstance(pop_value, str):
                    pop_value = pop_value.replace(',', '').strip()
                try:
                    pop_value = float(pop_value)
                    if pop_value > 0:  # 유효한 값만 추가
                        population_data.append({
                            'region': region.strip(),
                            'year': year,
                            'population': pop_value
                        })
                except:
                    pass

pop_clean = pd.DataFrame(population_data)
print(f" 인구 데이터 전처리 완료: {len(pop_clean)}개 레코드")
print(f"   연도 범위: {pop_clean['year'].min()} - {pop_clean['year'].max()}")
print(f"   구 개수: {pop_clean['region'].nunique()}개")

# ===============================
# 3. 범죄 데이터 확인 및 병합
# ===============================
print("\n 범죄 데이터 컬럼:", crime_df.columns.tolist())


merged_df = pd.merge(crime_df, pop_clean, on=['region', 'year'], how='inner')
merged_df['crime_rate'] = (merged_df['total'] / merged_df['population']) * 100000
merged_df['crime_rate'] = merged_df['crime_rate'].round(2)

print(f" 범죄율 계산 완료: {len(merged_df)}개 레코드")

# ===============================
# 4. 데이터 정렬 및 저장
# ===============================
merged_df_sorted = merged_df.sort_values(['year', 'region']).reset_index(drop=True)

output_file = 'crime_rate_long.csv'
merged_df_sorted[['region', 'year', 'population', 'total', 'crime_rate']].to_csv(
    output_file,
    index=False,
    encoding='utf-8-sig'
)

print(f"\n CSV 저장 완료: {output_file}")
print("   형식: region, year, population, total, crime_rate")

# ===============================
# 5. 미리보기
# ===============================
print("\n" + "="*60)
print(" 저장된 데이터 미리보기")
print("="*60)
print(merged_df_sorted[['region', 'year', 'crime_rate']].head(10))
print("\n연도별 레코드 수:")
print(merged_df_sorted.groupby('year').size())

print(f"\n {output_file} 파일이 생성되었습니다.")

데이터 로드 완료
 인구 데이터 전처리 완료: 275개 레코드
   연도 범위: 2014 - 2024
   구 개수: 25개

 범죄 데이터 컬럼: ['region', 'year', 'total']
 범죄율 계산 완료: 264개 레코드

 CSV 저장 완료: crime_rate_long.csv
   형식: region, year, population, total, crime_rate

 저장된 데이터 미리보기
  region  year  crime_rate
0    강남구  2014     3728.70
1    강동구  2014     2900.41
2    강북구  2014     2853.18
3    강서구  2014     2256.61
4    관악구  2014     2732.29
5    광진구  2014     3953.98
6    구로구  2014     3122.36
7    금천구  2014     3696.36
8    노원구  2014     2402.46
9    도봉구  2014     2280.67

연도별 레코드 수:
year
2014    24
2015    24
2016    24
2017    24
2018    24
2019    24
2020    24
2021    24
2022    24
2023    24
2024    24
dtype: int64

 crime_rate_long.csv 파일이 생성되었습니다.
